In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [2]:
model_local = ChatOllama(model="mistral",temperature=0)

In [3]:
loader = BSHTMLLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\check.html",open_encoding='utf-8')
data = loader.load()

In [4]:
data

[Document(page_content='Studies have shown that a woman’s risk of developing breast cancer is related to her exposure to hormones that are produced by her ovaries (endogenous estrogen and progesterone). Reproductive factors that increase the duration and/or levels of exposure to ovarian hormones, which stimulate cell growth, have been associated with an increase in breast cancer risk. These factors include early onset of menstruation, late onset of menopause, and factors that may allow breast tissue to be exposed to high levels of hormones for longer periods of time, such as later age at first pregnancy and never having given birth.', metadata={'source': 'C:\\Users\\Barani\\Desktop\\local_ollama\\check.html', 'title': ''})]

In [11]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=100)
doc_splits = text_splitter.split_documents(data)

In [12]:
len(doc_splits)

1

In [13]:
from langchain_community.vectorstores import FAISS

In [14]:
embeddings = embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text')
db = FAISS.from_documents(doc_splits, embeddings)
print(db.index.ntotal)
# db.save_local("cancer_index")

1


In [15]:
query = "Is there a relationship between pregnancy and breast cancer risk?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Studies have shown that a woman’s risk of developing breast cancer is related to her exposure to hormones that are produced by her ovaries (endogenous estrogen and progesterone). Reproductive factors that increase the duration and/or levels of exposure to ovarian hormones, which stimulate cell growth, have been associated with an increase in breast cancer risk. These factors include early onset of menstruation, late onset of menopause, and factors that may allow breast tissue to be exposed to high levels of hormones for longer periods of time, such as later age at first pregnancy and never having given birth.
